In [13]:
## run for the first time
# import nltk; nltk.download('stopwords')

import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy
nlp = spacy.load("en_core_web_sm")

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# sentiment analysis
import json
import string
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import collections

In [14]:
# NLTK Stop words
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'idea', 'proposal', 'plastic'])

In [15]:
# Import Dataset
# df = pd.read_json('data_ccrit.json')
df = pd.read_json('data_envcanada.json')
df.head()

,idea,desctiption,task,comment
0,Sustainable Design and Production,What factors do you consider when you purchase...,Share your comment,"1)quite often, will not purchase something tha..."
1,Sustainable Design and Production,What factors do you consider when you purchase...,Share your comment,"Yes we need to reduce plastics, but why are t..."
2,Sustainable Design and Production,What factors do you consider when you purchase...,Share your comment,"Everyone talks about plastics being ""recyclabl..."
3,Sustainable Design and Production,What factors do you consider when you purchase...,Share your comment,1. The amount and need for packaging. \n2. Pla...
4,Sustainable Design and Production,What factors do you consider when you purchase...,Share your comment,These are little ideas: I am at a loss when I ...


In [16]:
# get proposal wise comments
ideas = df.idea.unique()
proposal_wise_comments = []

for idea in ideas:
    proposal_wise_comments.append(list(df.loc[df['idea'] == idea, 'comment']))

In [17]:
### Preprocessing

# cleanup
def cleanup(data):
    # Remove Emails
    data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
    
    # Remove new line characters
    data = [re.sub('\s+', ' ', sent) for sent in data]

    # Remove distracting single quotes
    data = [re.sub("\'", "", sent) for sent in data]
    
    return data

# tokenize
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations


def create_gram_models(data_words):
    # Build the bigram and trigram models
    bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
    trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

    # Faster way to get a sentence clubbed as a trigram/bigram
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)

#     # See trigram example
#     print(trigram_mod[bigram_mod[data_words[0]]])
    
    return [bigram_mod, trigram_mod]

# functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [18]:
# model generation

def create_dict_and_corpus(data_lemmatized):
    # Create Dictionary
    id2word = corpora.Dictionary(data_lemmatized)

    # Create Corpus
    texts = data_lemmatized

    # Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in texts]

    # # Human readable format of corpus (term-frequency)
    # [[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]
    
    return id2word, corpus

# create lda model (mallet)
def create_mallet_lda_model(id2word, corpus):
    # # Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
    import os
    os.environ.update({'MALLET_HOME': r'/home/mjasim/Matryona/topic_modeling/mallet/'})
    mallet_path = '/home/mjasim/Matryona/topic_modeling/mallet/bin/mallet'
    ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=10, id2word=id2word)
    
    return ldamallet

In [19]:
# Compute Coherence Score
def get_coherence_score(ldamallet, data_lemmatized, id2word):
    coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    coherence_ldamallet = coherence_model_ldamallet.get_coherence()
    return coherence_ldamallet

In [20]:
# get optimal number of topics

def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    
    mallet_path = '/home/mjasim/Matryona/topic_modeling/mallet/bin/mallet'
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)        
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [21]:
# find the best model
def get_best_model(model_list, coherence_values, start, limit, step):
    max = 0
    num_topics = 0
    list_idx = 0
    result = []
    x = range(start, limit, step)
    for m, cv in zip(x, coherence_values):
#         print("Num Topics =", m, " has Coherence Value of", round(cv, 4))
        if(cv > max):
            max = cv
            num_topics = m
            model_idx = list_idx
        list_idx += 1

    # Select the best model
    optimal_model = model_list[model_idx]
    
    return optimal_model

In [22]:
### find dominant topic in each sentence

def format_topics_sentences(ldamodel, corpus, texts):
    # Init output
    sent_topics_df = pd.DataFrame()
    
    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

#     # Add original text to the end of the output
#     contents = pd.Series(texts)
#     sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [28]:
# pipeline

frames = []

for data in proposal_wise_comments:
    data_words = cleanup(data)        
    data_words = list(sent_to_words(data))
    bigram_mod, trigram_mod = create_gram_models(data_words)
    data_words_nostops = remove_stopwords(data_words)
    data_words_bigrams = make_bigrams(data_words_nostops)
    nlp = spacy.load('en', disable=['parser', 'ner'])
    data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
#     print(data_lemmatized)
    id2word, corpus = create_dict_and_corpus(data_lemmatized)
    ldamallet = create_mallet_lda_model(id2word, corpus)
    coherence_ldamallet = get_coherence_score(ldamallet, data_lemmatized, id2word)
    start = 2
    limit = 30
    step = 2
    model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=start, limit=limit, step=step)
    optimal_model = get_best_model(model_list, coherence_values, start, limit, step)
    df_topic_sents_keywords = format_topics_sentences(optimal_model, corpus, data)
    # Format
    df_dominant_topic = df_topic_sents_keywords.reset_index()
    df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'topic_perc_contrib', 'keywords']
    frames.append(df_dominant_topic)

sent_wise_df = pd.concat(frames)

[['quite', 'often', 'purchase', 'pack', 'especially', 'regrettably', 'clothing', 'make', 'polymer', 'add', 'problem', 'recycling', 'really', 'available', 'end', 'life', 'item', 'need', 'reinvent', 'wheel', 'country', 'already', 'alter', 'packaging', 'biodegradable', 'look', 'technology', 'already', 'establish', 'improve', 'packaging', 'extend', 'life', 'alter', 'problem'], ['need', 'reduce', 'plastic', 'barrier', 'recycle', 'need', 'cup', 'paper', 'one', 'work', 'bag', 'issue', 'many', 'case', 'necessary', 'however', 'paper', 'bag', 'work', 'foam', 'packaging', 'ban', 'protection', 'work', 'fine', 'big', 'issue', 'inefficient', 'recycling', 'program', 'many', 'must', 'not', 'secondly', 'entire', 'process', 'collection', 'total', 'nonsense', 'finally', 'plant', 'process', 'glass', 'plastic', 'metal', 'stuff', 'send'], ['talk', 'plastic', 'recyclable', 'yet', 'many', 'packaging', 'product', 'type', 'fact', 'fit', 'meet', 'recyclable', 'standard', 'fall', 'garbage', 'category', 'consumer'

KeyboardInterrupt: 

In [ ]:
pd.set_option('display.max_rows', None)
# pd.set_option('display.max_colwidth', -1)

# display(sent_wise_df)

In [ ]:
x = df
x = x.reset_index(drop=True)
sent_wise_df = sent_wise_df.reset_index(drop=True)

x = x.join(sent_wise_df)

x = x.rename(columns={"desctiption": "description", "topic_Perc_Contrib": "topic_perc_contrib", "Dominant_Topic": "dominant_topic"})
# topic_df = x[['idea', 'description', 'task', 'comment', 'reaction', 'topic_perc_contrib', 'keywords']].copy()
topic_df = x[['idea', 'description', 'task', 'comment', 'dominant_topic', 'topic_perc_contrib', 'keywords']].copy()
topic_df

In [ ]:
# load pos and neg lexicons

with open('pos_lex.tex') as json_file:
    pos = json.load(json_file)
    
with open('neg_lex.tex') as json_file:
    neg = json.load(json_file)
    
pos = [p.lower() for p in pos]
neg = [n.lower() for n in neg]

In [ ]:
sentiment_array = []
sentiment_score = []
for i in topic_df['comment']:
    pos_score = 0
    neg_score = 0
    hit_count = 0
    i = i.translate(str.maketrans('','', string.punctuation))
    i = re.sub(r"\s+", " ", i)
    tokens = word_tokenize(i)
    tokens = [t.lower() for t in tokens]
#     tokens = [t for t in tokens if t not in stopwords.words('english')]
    for t in tokens:
        if (t in pos):
            pos_score += 1
            hit_count += 1
        elif (t in neg):
            neg_score -= 1
            hit_count += 1
    
    if(hit_count > 0):
        score = (0.7 * neg_score + 0.3 * pos_score) / hit_count
    else:
        score = 0
    
    if score <= 0:
        sentiment_array.append('negative') 
    else:
        sentiment_array.append('positive')
        
    sentiment_score.append(score)

collections.Counter(sentiment_array)

In [ ]:
topic_df['sentiment'] = sentiment_array
topic_df['sentiment_score'] = sentiment_score

topic_df

In [ ]:
# presentation

show_sentence = [None] * len(df)
nlp = spacy.load("en_core_web_sm")

for i, row in topic_df.iterrows():
    doc = nlp(row['comment'])
    max_hit_sentence = 0
    sentences = list(doc.sents)
    for sentence in sentences:
        sentence = str(sentence)   
        pos_score = 0
        neg_score = 0
        hit_count = 0
        tokens = nlp(sentence)
        tokens = [t.text for t in tokens]
        tokens = [t.lower() for t in tokens]
        
        if (row['sentiment'] == 'positive'):
            for t in tokens:
                if (t in pos):
                    hit_count += 1
        elif (row['sentiment'] == 'negative'):
            for t in tokens:
                if (t in pos):
                    hit_count += 1
                    
        if(hit_count >= max_hit_sentence):
            max_score_sentence = score
            best_sentence = sentence
            
    show_sentence[i] = best_sentence

In [ ]:
topic_df['presentation'] = show_sentence

topic_df

In [ ]:
# topic_df.to_json("cc_sent.json", orient="records")
topic_df.to_json("env_sent.json", orient="records")

In [ ]:
#scrap

df = pd.read_json('cc_sent.json')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)
display(df[['comment','sentiment']])

In [ ]:
df = pd.read_json('env_sent.json')

df

In [ ]:
# # scrap

# import pandas as pd

# # df = pd.read_json('cc_sent.json')
# df = pd.read_json('env_sent.json')

# new_df = df[['comment', 'sentiment']].copy()

# temp = []
# for i in new_df['sentiment']:
#     if(i == 'positive'):
#         temp.append(1)
#     else:
#         temp.append(0)
        
# new_df['sentiment'] = temp

# new_df = new_df.rename(columns={"comment": "sentence", "sentiment": "polarity"})

# # new_df.to_json("cc_bert.json", orient="records")
# new_df.to_json("env_bert.json", orient="records")